<a href="https://colab.research.google.com/github/AlexFly666/LLM-in-Practice/blob/main/chapter09/0_quikstart/rag_react_agent_chroma_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# LangChain 最新版本
# !pip install langchain \
#     langchain-community \
#     langchain-core \
#     langchain-openai
# LangChain V0.3版本
!pip install langchain==0.3.18 \
    langchain-community==0.3.17 \
    langchain-core==0.3.34 \
    langchain-openai==0.3.4

In [ ]:
!pip show langchain \
    langchain-community \
    langchain-core \
    langchain-openai

Name: langchain
Version: 0.3.18
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
---
Name: langchain-community
Version: 0.3.17
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.3.34
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: jsonpatch, langsmith, packaging, pydantic, PyYAML, tenacity, typing

In [31]:
# 导入操作系统库，用于文件路径操作等
import os

# 导入Langchain库中的HuggingFaceEmbeddings类，用于使用HuggingFace模型生成文本嵌入
from langchain.embeddings import HuggingFaceEmbeddings,HuggingFaceBgeEmbeddings
# 导入Langchain库中的CharacterTextSplitter类，用于按字符分割文本
from langchain.text_splitter import CharacterTextSplitter
# 导入Langchain Community库中的TextLoader类，用于加载文本文件
from langchain_community.document_loaders import TextLoader
# 导入LangchainCommunity库中的Chroma类，用于创建和管理Chroma向量数据库
from langchain_community.vectorstores import Chroma
# 导入Langchain OpenAI库中的OpenAIEmbeddings类，用于使用OpenAI模型生成文本嵌入
from langchain_openai import OpenAIEmbeddings

# 获取当前脚本文件所在的目录
current_dir = os.getcwd()
# 构建书籍文件（odyssey.txt）的完整路径，假设书籍文件位于当前目录下的 "books" 文件夹中
file_path = os.path.join(current_dir, "data", "books", "xiyouji.txt")
# 构建持久化向量数据库的目录路径，数据库文件将存储在当前目录下的 "db" 文件夹中
db_dir = os.path.join(current_dir, "db")

# 检查书籍文件是否存在
if not os.path.exists(file_path):
    # 如果书籍文件不存在，则抛出 FileNotFoundError 异常，提示用户检查文件路径
    raise FileNotFoundError(
        f"The file {file_path} does not exist. Please check the path."
    )

# 使用 TextLoader 加载文本文件内容
loader = TextLoader(file_path)
# 将加载的文本内容存储在documents变量中，TextLoader将文件内容加载为Document对象列表
documents = loader.load()

# 创建CharacterTextSplitter实例，用于将文档分割成更小的文本块
# chunk_size=1000 表示每个文本块的大小为1000个字符
# chunk_overlap=0 表示文本块之间没有重叠部分
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# 使用text_splitter将加载的文档分割成文本块，存储在docs变量中，docs是Document对象列表
docs = text_splitter.split_documents(documents)

# 打印文档分割块的信息
print("\n--- Document Chunks Information ---")
# 打印分割后的文档块数量
print(f"Number of document chunks: {len(docs)}")
# 打印第一个文档块的内容示例，用于预览分割效果
# print(f"Sample chunk:\n{docs[0].page_content}\n")


# 定义函数create_vector_store，用于创建并持久化向量数据库
# 参数 docs：文档块列表，embeddings：嵌入模型，store_name：向量数据库存储名称
def create_vector_store(docs, embeddings, store_name):
    # 构建持久化向量数据库的完整目录路径
    persistent_directory = os.path.join(db_dir, store_name)
    # 检查持久化目录是否已存在
    if not os.path.exists(persistent_directory):
        # 如果持久化目录不存在，则表示需要创建新的向量数据库
        print(f"\n--- Creating vector store {store_name} ---")
        # 使用 Chroma.from_documents 方法从文档块创建向量数据库
        # docs：文档块列表，embeddings：嵌入模型，persist_directory：持久化目录
        Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory)
        # 打印向量数据库创建完成的提示信息
        print(f"--- Finished creating vector store {store_name} ---")
    else:
        # 如果持久化目录已存在，则表示向量数据库已存在，无需重新初始化
        print(
            f"Vector store {store_name} already exists. No need to initialize.")


# 1. OpenAI Embeddings
# 使用 OpenAI 的嵌入模型。
# 适用于通用目的的嵌入，具有高准确性。
# 注意：使用 OpenAI 嵌入的成本取决于您的 OpenAI API 使用量和定价计划。
# 定价：https://openai.com/api/pricing/
# print("\n--- Using OpenAI Embeddings ---")
# # 创建 OpenAIEmbeddings 实例，使用 "text-embedding-ada-002" 模型
# # openai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# openai_embeddings = OpenAIEmbeddings(
#     api_key="sk-paSHgQoVeKag1rou9d81Fa2f534940C1Ba394f02C45aF3D2",
#     base_url="https://vip.apiyi.com/v1",
#     model="text-embedding-3-small"
# )
# # 调用create_vector_store函数，使用OpenAI嵌入模型创建名为 "chroma_db_openai" 的向量数据库
# create_vector_store(docs, openai_embeddings, "chroma_db_openai")

# 2. Hugging Face Transformers
# 使用 Hugging Face 库的模型。
# 非常适合利用各种模型来完成不同的任务。
# 注意：在您的机器本地运行 Hugging Face模型除了使用您的计算资源外，没有其他直接成本。
# 注意：在 https://huggingface.co/models?other=embeddings 查找其他模型
print("\n--- Using Hugging Face Transformers ---")
# 创建 HuggingFaceEmbeddings 实例，使用 "sentence-transformers/all-mpnet-base-v2" 模型
# huggingface_embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2"
# )
model_name = "BAAI/bge-large-zh-v1.5"
# 根据你的需要去选择设备
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
# 调用create_vector_store函数，使用Hugging Face嵌入模型创建名为 "chroma_db_huggingface" 的向量数据库
create_vector_store(docs, huggingface_embeddings, "chroma_db_huggingface")

# 打印嵌入演示完成的提示信息
print("Embedding demonstrations for OpenAI and Hugging Face completed.")


# 定义函数query_vector_store，用于查询向量数据库
# 参数 store_name：向量数据库存储名称，query：查询语句，embedding_function：嵌入函数
def query_vector_store(store_name, query, embedding_function):
    # 构建持久化向量数据库的完整目录路径
    persistent_directory = os.path.join(db_dir, store_name)
    # 检查持久化目录是否已存在，以确保向量数据库存在
    if os.path.exists(persistent_directory):
        # 如果持久化目录存在，则表示向量数据库存在，可以进行查询
        print(f"\n--- Querying the Vector Store {store_name} ---")
        # 加载已持久化的向量数据库
        db = Chroma(
            persist_directory=persistent_directory,
            embedding_function=embedding_function,
        )
        # 创建检索器，用于从向量数据库中检索相关文档
        retriever = db.as_retriever(
            search_type="similarity_score_threshold",
            # search_kwargs 参数用于设置检索参数
            # k=3 表示检索 top 3 个最相关的文档
            # score_threshold=0.1 表示相似度得分阈值为 0.1，只有相似度得分高于 0.1 的文档才会被检索出来
            search_kwargs={"k": 3, "score_threshold": 0.001},
        )
        # 使用检索器根据查询语句检索相关文档
        relevant_docs = retriever.invoke(query)
        # 打印相关文档的标题
        print(f"\n--- Relevant Documents for {store_name} ---")
        # 遍历检索到的相关文档，并打印文档内容和元数据
        for i, doc in enumerate(relevant_docs, 1):
            # 打印文档内容
            print(f"Document {i}:\n{doc.page_content}\n")
            # 检查文档是否包含元数据
            if doc.metadata:
                # 打印文档来源，如果元数据中包含 'source' 字段，则打印来源信息，否则打印 'Unknown'
                print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")
    else:
        # 如果持久化目录不存在，则表示向量数据库不存在，打印提示信息
        print(f"Vector store {store_name} does not exist.")


# 定义用户的查询问题
query = "孙悟空的师傅是谁?"

# 使用 OpenAI 嵌入查询 "chroma_db_openai" 向量数据库
# query_vector_store("chroma_db_openai", query, openai_embeddings)
# 使用 Hugging Face 嵌入查询 "chroma_db_huggingface" 向量数据库
query_vector_store("chroma_db_huggingface", query, huggingface_embeddings)

# 打印查询演示完成的提示信息
print("Querying demonstrations completed.")


--- Document Chunks Information ---
Number of document chunks: 1

--- Using Hugging Face Transformers ---

--- Creating vector store chroma_db_huggingface ---


InvalidDimensionException: Embedding dimension 1024 does not match collection dimensionality 768

这段代码实现了一个**具备对话历史记录功能的检索增强生成 (RAG) 聊天机器人**。 简单来说，它是一个可以和你**像真人一样聊天**的 AI 助手，并且它还很聪明，因为：

1.  **它能记住你们之前的对话:**  不像普通的聊天机器人，它不会忘记你之前说过什么。如果你问了一个问题，然后在之后的问题里用 "它" 或 "他" ，它也能明白你在指代什么，因为它记住了对话的上下文。 **[解决问题：理解对话上下文，使多轮对话更自然]**  **[达到效果：更流畅、更连贯的对话体验]**

2.  **它懂得很多东西，而且知识可以更新:**  它不仅仅依靠它自己训练时学到的知识，更重要的是，它能连接到一个**知识库 (就像一个图书馆)**，从中查找信息来回答你的问题。 如果知识库更新了，它的知识也会跟着更新，不会过时。 **[解决问题：语言模型知识有限，无法回答特定领域或最新信息问题]**  **[达到效果：能够回答更广泛、更专业、更及时的提问]**

3.  **它用检索到的信息来生成答案，所以答案更可靠:**  当它回答问题时，不是随便乱说，而是会先从知识库里找到相关的资料，然后**基于这些资料来组织答案**。 这就像写论文时引用参考文献一样，答案更有依据，也更可信。  **[解决问题：生成答案缺乏依据，可能不准确或不可靠]**  **[达到效果：提高答案的准确性和可信度]**

**总结来说，这段代码就像是打造了一个聪明的 AI 助手，它通过记住对话、查找资料、并基于资料生成答案，从而能够更好地理解你的问题并给出有用的回答。**

**代码主要做了以下事情：**

*   **加载知识库:**  代码连接到一个已经准备好的知识库 (向量数据库)，这个知识库里存储了很多信息，等待被检索和利用。
*   **创建智能检索工具:**  代码创建了一个 "历史感知检索器"，这个工具可以理解你问题的含义，包括问题中可能包含的上下文信息，并在知识库中找到最相关的资料。
*   **创建智能问答工具:**  代码创建了一个 "问答链"，这个工具会利用找到的资料，结合强大的语言模型 (例如 gpt-4o)，来生成简洁明了的答案。
*   **连接检索和问答工具:**  代码将检索工具和问答工具连接起来，形成一个完整的 **RAG 聊天机器人系统**。
*   **实现持续对话功能:**  代码提供了一个简单的聊天界面，你可以通过输入文字与 AI 助手进行多轮对话。

**最终达到的效果：**

*   **用户可以与 AI 进行自然的、有上下文的对话。**
*   **AI 能够回答各种基于知识库的问题，即使是复杂或需要专业知识的问题。**
*   **AI 的答案简洁、明了、并且有信息来源支撑，更可靠。**

**解决的主要问题：**

*   **传统聊天机器人无法理解对话上下文的问题。**
*   **语言模型自身知识有限，无法回答特定领域或最新信息的问题。**
*   **生成答案缺乏依据，可能不准确或不可靠。**
*   **构建复杂的 RAG 聊天机器人流程比较繁琐。** （代码通过使用 Langchain 框架简化了构建过程）

希望这个更简洁的说明能够帮助你更好地理解代码的功能、效果以及解决的问题! 如果你想了解更详细的解释，可以回顾一下之前更长的回复，里面有更深入的分析和例子。

In [ ]:
# 导入操作系统库，用于处理文件路径等操作系统相关的功能
import os


# 导入 langchain 库中的 create_history_aware_retriever 和 create_retrieval_chain 函数，用于创建支持历史记录的检索器和检索链
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# 导入 langchain 库中的 create_stuff_documents_chain 函数，用于创建将所有检索到的文档塞入单个上下文的文档组合链
from langchain.chains.combine_documents import create_stuff_documents_chain
# 导入 langchain_community.vectorstores 库中的 Chroma 类，用于使用 Chroma 向量数据库
from langchain_community.vectorstores import Chroma
# 导入 langchain_core.messages 库中的 HumanMessage 和 SystemMessage 类，用于构建对话消息
from langchain_core.messages import HumanMessage, SystemMessage
# 导入 langchain_core.prompts 库中的 ChatPromptTemplate 和 MessagesPlaceholder 类，用于创建聊天提示模板和消息占位符
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 导入 langchain_openai 库中的 ChatOpenAI 和 OpenAIEmbeddings 类，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI, OpenAIEmbeddings



# 定义持久化目录
# 获取当前文件所在目录
current_dir = os.getcwd()
# 构建Chroma数据库的持久化目录路径，位于当前目录下的 db/chroma_db_with_metadata 文件夹
persistent_directory = os.path.join(current_dir, "db", "chroma_db_with_metadata")

# 定义嵌入模型
# 使用 OpenAIEmbeddings 创建嵌入模型实例，模型为 "text-embedding-3-small"
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 词嵌入（代理方式）
embeddings_model = OpenAIEmbeddings(
    api_key="sk-paSHgQoVeKag1rou9d81Fa2f534940C1Ba394f02C45aF3D2",
    base_url="https://vip.apiyi.com/v1",
    model="text-embedding-3-small"
)


# 加载已存在的向量存储
# 使用 Chroma类加载持久化目录中的向量数据库，并使用之前定义的嵌入模型
db = Chroma(persist_directory=persistent_directory, embedding_function=embeddings)

# 创建检索器
# 使用 as_retriever 方法从 Chroma 数据库创建检索器
# search_type="similarity" 指定使用相似度搜索
# search_kwargs={"k": 3} 指定搜索时返回最相似的 3 个文档
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

# 创建 ChatOpenAI 模型
# 使用 ChatOpenAI 类创建聊天模型实例，模型为 "gpt-4o"
llm = ChatOpenAI(model="gpt-4o")

# 上下文化问题提示
# 定义系统提示，用于指示 AI 根据聊天历史和最新的用户问题，生成一个无需聊天历史也能理解的独立问题
# 解决问题：当用户的问题依赖于上下文时，确保 AI 可以正确理解用户最新的问题
# 达到的效果：使 AI 能够处理带有上下文依赖的问题，提升对话的连贯性
contextualize_q_system_prompt = (
    "给定聊天历史和最新的用户问题，"
    "最新的用户问题可能引用聊天历史中的上下文，"
    "请构建一个可以独立理解的问题，无需聊天历史。"
    "不要回答问题，只需在需要时改述问题，否则按原样返回。"
)

# 创建用于上下文化的提示模板
# 使用 ChatPromptTemplate.from_messages 创建聊天提示模板
# 包括系统消息（contextualize_q_system_prompt）、消息占位符（chat_history）和人类消息（input）
# 解决问题：结构化提示，方便传入聊天历史和用户输入，并指示 AI 执行上下文理解和问题改述
# 达到的效果：定义了 AI 如何接收和处理上下文信息以生成独立问题
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 创建历史感知检索器
# 使用 create_history_aware_retriever 函数创建历史感知检索器
# 参数包括聊天模型 (llm)、检索器 (retriever) 和上下文提示模板 (contextualize_q_prompt)
# 解决问题：结合聊天模型和检索器，使检索过程能够理解和利用对话历史
# 达到的效果：检索器不仅能基于当前问题检索，还能考虑之前的对话内容，提高检索的准确性
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# 回答问题提示
# 定义系统提示，用于指示 AI 使用检索到的上下文来回答问题，并限制答案的长度和处理未知答案的情况
# 解决问题：指导 AI 如何基于检索到的信息生成答案，并处理知识库中没有答案的情况
# 达到的效果：确保 AI 能够根据提供的上下文给出简洁明了的答案，并在无法回答时给出明确的提示
qa_system_prompt = (
    "你是一个问答任务的助手。 使用"
    "以下检索到的上下文片段来回答问题。 如果你不知道答案，就说你"
    "不知道。 答案最多使用三句话，并保持答案简洁。"
    "\n\n"
    "{context}"
)

# 创建用于回答问题的提示模板
# 使用 ChatPromptTemplate.from_messages 创建聊天提示模板
# 包括系统消息（qa_system_prompt）、消息占位符（chat_history）和人类消息（input）
# 解决问题：结构化提示，方便传入上下文信息、聊天历史和用户输入，并指示 AI 如何生成答案
# 达到的效果：定义了 AI 如何接收上下文并生成最终答案
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# 创建用于组合文档以进行问答的链
# 使用 create_stuff_documents_chain 函数创建文档组合链
# 参数包括聊天模型 (llm) 和问答提示模板 (qa_prompt)
# 解决问题：将检索到的多个文档整合为一个上下文，方便 AI 理解和提取信息
# 达到的效果：AI 可以一次性处理所有相关的检索结果，并从中提取答案
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# 创建检索链
# 使用 create_retrieval_chain 函数创建检索链，将历史感知检索器和问答链组合起来
# 参数包括历史感知检索器 (history_aware_retriever) 和问答链 (question_answer_chain)
# 解决问题：将检索和问答两个核心步骤连接起来，形成一个完整的 RAG (Retrieval-Augmented Generation) 流程
# 达到的效果：构建了一个完整的问答系统，能够处理带有上下文的查询，检索相关文档，并生成答案
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# 模拟持续对话的函数
def continual_chat():
    print("开始与 AI 聊天! 输入 'exit' 结束对话。")
    chat_history = []  # 用于收集聊天历史记录 (消息序列)
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        # 通过检索链处理用户的查询
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # 显示 AI 的回复
        print(f"AI: {result['answer']}")
        # 更新聊天历史
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result["answer"]))


# 主函数，用于启动持续对话
if __name__ == "__main__":
    continual_chat()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
# 导入 os 模块，用于处理文件路径和目录
import os

# 从 langchain 库的 hub 模块导入 hub，用于访问 prompt 的 hub
from langchain import hub
# 从 langchain 库的 agents 模块导入 AgentExecutor 和 create_react_agent，用于创建和运行 ReAct 代理
from langchain.agents import AgentExecutor, create_react_agent
# 从 langchain 库的 chains 模块导入 create_history_aware_retriever 和 create_retrieval_chain，用于创建检索链
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
# 从 langchain 库的 chains.combine_documents 模块导入 create_stuff_documents_chain，用于创建 stuff 文档链
from langchain.chains.combine_documents import create_stuff_documents_chain
# 从 langchain_community 库的 vectorstores 模块导入 Chroma，用于使用 Chroma 向量数据库
from langchain_community.vectorstores import Chroma
# 从 langchain_core 库的 messages 模块导入 AIMessage 和 HumanMessage，用于表示 AI 消息和人类消息
from langchain_core.messages import AIMessage, HumanMessage
# 从 langchain_core 库的 prompts 模块导入 ChatPromptTemplate 和 MessagesPlaceholder，用于创建聊天提示模板和消息占位符
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 从 langchain_core 库的 tools 模块导入 Tool，用于创建工具
from langchain_core.tools import Tool
# 从 langchain_openai 库导入 ChatOpenAI 和 OpenAIEmbeddings，用于使用 OpenAI 的聊天模型和嵌入模型
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
#

"""
1. 配置 Chroma 向量数据库目录

定义 Chroma 向量数据库的持久化存储目录。
向量数据库用于存储和检索文档嵌入，以便进行语义搜索和 RAG (Retrieval-Augmented Generation)。
"""
# Load the existing Chroma vector store
# 获取当前脚本的目录
# current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = os.getcwd()
# 构建数据库目录的路径，假设数据库位于rag目录下的db目录中
db_dir = os.path.join(current_dir, "rag", "db")
# 构建 Chroma 数据库持久化存储的完整路径
persistent_directory = os.path.join(db_dir, "chroma_db_with_metadata")

"""
检查并加载现有的Chroma向量数据库

检查指定的持久化目录是否存在 Chroma 向量数据库。
如果存在，则加载现有的数据库；如果不存在，则抛出 FileNotFoundError 异常，提示用户检查路径。
这避免了每次运行代码时都重新创建向量数据库，提高了效率。
"""
# Check if the Chroma vector store already exists
if os.path.exists(persistent_directory):
    print("Loading existing vector store...")
    # 如果目录存在，则加载已有的Chroma向量数据库，embedding_function 初始设置为 None，稍后会更新
    db = Chroma(persist_directory=persistent_directory,
                embedding_function=None)
else:
    # 如果目录不存在，抛出异常，提示用户检查路径
    raise FileNotFoundError(
        f"The directory {persistent_directory} does not exist. Please check the path."
    )

"""
2. 定义嵌入模型

使用 OpenAIEmbeddings 定义文档嵌入模型。
嵌入模型用于将文本转换为向量，以便在向量空间中进行相似性搜索。
这里使用 'text-embedding-3-small' 模型，这是一个性价比高的嵌入模型。
"""
# Define the embedding model
# 创建 OpenAIEmbeddings 实例，使用 'text-embedding-3-small' 模型
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

"""
3. 重新加载 Chroma 向量数据库并配置嵌入函数

使用之前定义的嵌入模型重新加载 Chroma 向量数据库。
在第一次加载时，embedding_function 设置为 None，现在使用 OpenAIEmbeddings 对象进行更新。
确保向量数据库使用正确的嵌入函数进行操作。
"""
# Load the existing vector store with the embedding function
# 重新加载 Chroma 向量数据库，这次使用 OpenAIEmbeddings 作为嵌入函数
db = Chroma(persist_directory=persistent_directory,
            embedding_function=embeddings)

"""
4. 创建检索器 (Retriever)

从 Chroma 向量数据库创建检索器。
检索器用于根据用户查询在向量数据库中搜索相关文档。
`search_type="similarity"` 指定使用相似性搜索，`search_kwargs={"k": 3}` 指定返回最相似的 3 个文档。
"""
# Create a retriever for querying the vector store
# 从 Chroma 数据库创建检索器，用于执行相似性搜索
# `search_type` 指定搜索类型为 "similarity" (相似性搜索)
# `search_kwargs` 设置搜索参数，`k=3` 表示返回最相关的 3 个文档
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

"""
5. 创建 ChatOpenAI 模型

初始化 ChatOpenAI 模型，使用 'gpt-4o' 模型。
这是用于生成答案和处理自然语言对话的大型语言模型。
'gpt-4o' 是一个先进的多模态模型，具有强大的文本处理能力。
"""
# Create a ChatOpenAI model
# 初始化 ChatOpenAI 模型，使用 'gpt-4o' 模型
# llm = ChatOpenAI(model="gpt-4o")

# OpenAI API调用（代理方式）
llm = ChatOpenAI(
    api_key="XXX",
    base_url="https://vip.apiyi.com/v1",
    model="gpt-4o"
)


"""
6. 上下文情境化问题提示 (Contextualize Question Prompt)

定义一个系统提示，用于指导 AI 如何根据聊天历史记录和最新的用户问题，生成一个独立的、无需上下文也能理解的问题。
这个提示的目的是让 AI 能够处理在对话中可能出现的指代和省略，确保检索器总是能获得清晰明确的查询。
"""
# Contextualize question prompt
# 系统提示，用于指导 AI 根据聊天历史和用户问题，生成一个独立的、无需上下文也能理解的问题
# 目标是让 AI 能够处理对话中的上下文依赖，生成清晰的查询
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

"""
7. 创建上下文情境化问题提示模板

使用 ChatPromptTemplate.from_messages 创建提示模板，该模板包含系统提示、聊天历史记录的占位符和用户输入的占位符。
MessagesPlaceholder("chat_history") 用于在运行时动态地插入聊天历史记录。
"""
# Create a prompt template for contextualizing questions
# 使用 ChatPromptTemplate 创建提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

"""
8. 创建历史感知检索器 (History-Aware Retriever)

使用 create_history_aware_retriever 函数，将 LLM、基础检索器和上下文情境化问题提示模板组合在一起，创建一个历史感知检索器。
这个检索器能够在检索文档之前，先利用 LLM 和聊天历史记录来提炼用户的问题，从而提高检索的准确性。
"""
# Create a history-aware retriever
# 使用 create_history_aware_retriever 创建历史感知检索器
# 结合 LLM, 基础检索器和上下文情境化问题提示模板
# 使得检索器能够根据聊天历史记录来优化检索query
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

"""
9. 答案问题提示 (Answer Question Prompt)

定义一个系统提示，用于指导 AI 如何使用检索到的上下文来回答用户问题。
该提示指示 AI 作为一个问题解答助手，使用检索到的上下文，并在不知道答案时明确表示“我不知道”。
同时，限制答案长度为最多三句话，保持简洁。
"""
# Answer question prompt
# 系统提示，用于指导 AI 如何使用检索到的上下文来回答问题
# 指示 AI 作为问题解答助手，使用上下文，并在不知道答案时回答 "I don't know"
# 限制答案长度为最多三句话，保持简洁
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

"""
10. 创建答案问题提示模板

使用 ChatPromptTemplate.from_messages 创建答案问题提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符。
虽然答案问题提示模板中也包含了 `MessagesPlaceholder("chat_history")`，但在当前的 `create_stuff_documents_chain` 使用方式下，聊天历史记录实际上并没有直接被答案生成链使用。
这里保留 `MessagesPlaceholder("chat_history")` 可能是为了未来扩展，或者保持提示模板结构的一致性。
"""
# Create a prompt template for answering questions
# 使用 ChatPromptTemplate 创建答案问题提示模板，包含系统提示、聊天历史记录占位符和用户输入占位符
# 注意：虽然这里也包含了 chat_history 占位符，但在当前的 chain 结构中，答案生成链可能并没有直接使用 chat_history
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

"""
11. 创建文档组合链 (Combine Documents Chain)

使用 create_stuff_documents_chain 函数，将 LLM 和答案问题提示模板组合在一起，创建一个文档组合链。
`create_stuff_documents_chain` 使用 "stuff" 方式，将所有检索到的文档合并成一个字符串，一次性送入 LLM 进行处理。
这个链负责接收检索器返回的文档，并将它们与用户问题一起传递给 LLM，以生成最终答案。
"""
# Create a chain to combine documents for question answering
# 使用 create_stuff_documents_chain 创建文档组合链
# `create_stuff_documents_chain` 使用 "stuff" 方式，将所有检索到的文档一次性喂给 LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

"""
12. 创建 RAG 链 (Retrieval-Augmented Generation Chain)

使用 create_retrieval_chain 函数，将历史感知检索器和文档组合链组合在一起，创建 RAG 链。
RAG 链是整个问答系统的核心，它首先使用历史感知检索器检索相关文档，然后使用文档组合链和 LLM 根据检索到的文档生成答案。
"""
# Create a retrieval chain that combines the history-aware retriever and the question answering chain
# 使用 create_retrieval_chain 创建 RAG 链，将历史感知检索器和答案生成链组合起来
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)


"""
13. 设置带有文档存储检索器的 ReAct Agent (ReAct Agent with Document Store Retriever)

配置 ReAct Agent，使其能够利用文档存储进行检索和问题回答。
ReAct (Reason and Act) 是一种 agent 框架，它允许 agent 在思考 (Reason) 和执行动作 (Act) 之间交替进行，从而更有效地完成复杂任务。
这里使用预定义的 ReAct 提示 (react_docstore_prompt) 和一个工具 (Answer Question Tool)。
"""
# Set Up ReAct Agent with Document Store Retriever
# 加载 ReAct Docstore Prompt
react_docstore_prompt = hub.pull("hwchase17/react")

"""
14. 定义工具 (Tools)

创建一个工具列表，ReAct Agent 可以使用这些工具来执行不同的操作。
这里定义了一个名为 "Answer Question" 的工具，该工具使用之前创建的 RAG 链 (rag_chain) 来回答问题。
工具的描述 (description) 非常重要，ReAct Agent 会根据描述来决定何时以及如何使用工具。
"""
# 定义 Agent 可以使用的工具列表
tools = [
    Tool(
        name="Answer Question", # 工具名称，Agent 会根据名称来调用工具
        # 工具的函数，这里使用 rag_chain.invoke 来执行 RAG 链，回答问题
        func=lambda input, **kwargs: rag_chain.invoke(
            {"input": input, "chat_history": kwargs.get("chat_history", [])}
        ),
        # 工具的描述，描述工具的用途，ReAct Agent 会根据描述来决定何时使用该工具
        description="useful for when you need to answer questions about the context",
    )
]

"""
15. 创建 ReAct Agent

使用 create_react_agent 函数创建 ReAct Agent。
需要传入 LLM, 工具列表和 ReAct 提示模板。
ReAct Agent 会根据提示模板和可用的工具，以及用户的输入，自主决定执行哪些操作来完成任务。
"""
# Create the ReAct Agent with document store retriever
# 使用 create_react_agent 创建 ReAct Agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt,
)

"""
16. 创建 Agent 执行器 (Agent Executor)

使用 AgentExecutor.from_agent_and_tools 创建 Agent 执行器。
Agent 执行器是运行 Agent 的核心组件，它负责接收用户输入，调用 Agent 进行决策，执行工具，并返回最终结果。
`handle_parsing_errors=True` 用于处理 Agent 输出解析错误的情况，`verbose=True` 开启详细日志输出，方便调试。
"""
# 创建 AgentExecutor，用于运行 Agent
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
)

"""
17. 主聊天循环 (Main Chat Loop)

实现一个无限循环，模拟聊天对话。
用户可以输入查询，Agent 执行器会处理查询并返回 AI 的回复。
聊天历史记录 (chat_history) 会在每次对话后更新，以便 Agent 可以感知上下文。
输入 "exit" 可以结束循环。
"""
# 初始化聊天历史记录列表
chat_history = []
# 进入无限循环，模拟聊天对话
while True:
    # 获取用户输入
    query = input("You: ")
    # 如果用户输入 "exit"，则退出循环
    if query.lower() == "exit":
        break
    # 调用 agent_executor 的 invoke 方法，传入用户输入和聊天历史记录，获取 AI 的回复
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    # 打印 AI 的回复
    print(f"AI: {response['output']}")

    # 更新聊天历史记录，将用户消息和 AI 消息添加到列表中
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))

FileNotFoundError: The directory /content/rag/db/chroma_db_with_metadata does not exist. Please check the path.